In [118]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import json, ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Uploading, Exploring and Wranggling the Data

In [226]:
# Upload into a Dataframe
df = pd.read_excel('safegraph-spend-sample-q3-2022.xlsx')
df.head(10)

,PLACEKEY,PARENT_PLACEKEY,LOCATION_NAME,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,LONGITUDE,STREET_ADDRESS,CITY,...,RELATED_CROSS_SHOPPING_PHYSICAL_BRANDS_PCT,RELATED_CROSS_SHOPPING_ONLINE_MERCHANTS_PCT,RELATED_CROSS_SHOPPING_SAME_CATEGORY_BRANDS_PCT,RELATED_CROSS_SHOPPING_LOCAL_BRANDS_PCT,RELATED_WIRELESS_CARRIER_PCT,RELATED_STREAMING_CABLE_PCT,RELATED_DELIVERY_SERVICE_PCT,RELATED_RIDESHARE_SERVICE_PCT,RELATED_BUYNOWPAYLATER_SERVICE_PCT,RELATED_PAYMENT_PLATFORM_PCT
0,222-227@8gn-9sp-nqz,222-223@8gn-9sp-nqz,Ross Stores,Department Stores,Department Stores,452210,33.185004,-87.527936,2600 McFarland Blvd E,Tuscaloosa,...,"{\n ""CVS"": 27,\n ""Chevron"": 55,\n ""Chick-fi...","{\n ""AT&T"": 24,\n ""Amazon"": 39,\n ""Amazon D...","{\n ""Bargain Hunt"": 2,\n ""Bealls Tx"": 1,\n ...","{\n ""Academy Sports + Outdoors"": 7,\n ""Chick...","{\n ""AT&T"": 33,\n ""Boost Mobile"": 1,\n ""Cri...","{\n ""Amazon Prime"": 19,\n ""Amazon Prime Vide...","{\n ""Dashpass"": 1,\n ""DoorDash"": 3,\n ""Grub...","{\n ""Lyft"": 5,\n ""Uber"": 10\n}","{\n ""Affirm.com"": 3,\n ""Afterpay"": 5,\n ""Kl...","{\n ""Apple Card"": 1,\n ""Apple Cash"": 1,\n ""..."
1,223-222@8f3-x8h-hkf,NaN,Cracker Barrel,Restaurants and Other Eating Places,Full-Service Restaurants,722511,31.353805,-89.347748,6659 U S Highway 49,Hattiesburg,...,"{\n ""Burger King"": 23,\n ""Chevron"": 42,\n ""...","{\n ""AT&T"": 25,\n ""Amazon"": 48,\n ""Amazon D...","{\n ""Burger King"": 23,\n ""Chick-fil-A"": 50,\...","{\n ""Burger King"": 6,\n ""Dairy Queen"": 5,\n ...","{\n ""AT&T"": 35,\n ""MetroPCS"": 1,\n ""Sprint""...","{\n ""Amazon Prime"": 24,\n ""Amazon Prime Vide...","{\n ""Dashpass"": 2,\n ""DoorDash"": 3,\n ""Hell...","{\n ""Lyft"": 4,\n ""Uber"": 9\n}","{\n ""Affirm.com"": 3,\n ""Afterpay"": 1,\n ""Kl...","{\n ""Apple Cash"": 1\n}"
2,zzw-222@8gf-ddy-6p9,NaN,Which Wich,Restaurants and Other Eating Places,Limited-Service Restaurants,722513,35.099116,-80.988821,14318 Rivergate View Dr,Charlotte,...,"{\n ""BP"": 45,\n ""Bojangles'"": 35,\n ""CVS"": ...","{\n ""AT&T"": 28,\n ""Amazon"": 54,\n ""Amazon D...","{\n ""Bojangles'"": 35,\n ""Burger King"": 25,\n...","{\n ""7-Eleven"": 6,\n ""Bojangles'"": 6,\n ""Ch...","{\n ""AT&T"": 35,\n ""Boost Mobile"": 1,\n ""Cri...","{\n ""Amazon Prime"": 15,\n ""DirecTV"": 3,\n ""...","{\n ""DoorDash"": 5,\n ""Go Puff"": 1,\n ""Hello...","{\n ""Lyft"": 6,\n ""Uber"": 13\n}","{\n ""Affirm.com"": 1,\n ""Klarna"": 1\n}","{\n ""Apple Card"": 2,\n ""Apple Cash"": 1,\n ""..."
3,223-222@5s5-gzd-6c5,NaN,Marathon,Gasoline Stations,Gasoline Stations with Convenience Stores,447110,42.841443,-85.577585,3495 68th St SE,Dutton,...,"{\n ""BP"": 29,\n ""Burger King"": 22,\n ""CITGO...","{\n ""AT&T"": 14,\n ""Amazon"": 49,\n ""Amazon M...","{\n ""Admiral"": 1,\n ""Admiral Tobacco"": 1,\n ...","{\n ""Applebee's"": 5,\n ""BP"": 8,\n ""D&W Fres...","{\n ""AT&T"": 17,\n ""Boost Mobile"": 3,\n ""Cri...","{\n ""Amazon Prime"": 23,\n ""Dish Network"": 2,...","{\n ""HelloFresh"": 2,\n ""Instacart"": 2,\n ""S...","{\n ""Lyft"": 4,\n ""Uber"": 11\n}","{\n ""Affirm.com"": 3,\n ""Afterpay"": 2,\n ""Kl...","{\n ""Apple Card"": 1,\n ""Cash App"": 1,\n ""Pa..."
4,zzw-223@8fz-thj-q75,225-222@8fz-thj-q75,Starbucks,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515,26.840002,-80.106207,10925 N Military Trl,Palm Beach Gardens,...,"{\n ""7-Eleven"": 34,\n ""CVS"": 56,\n ""Chevron...","{\n ""AT&T"": 15,\n ""Amazon"": 50,\n ""Amazon D...","{\n ""Burger King"": 13,\n ""Chick-fil-A"": 40,\...","{\n ""Barnes and Noble"": 7,\n ""CVS"": 12,\n ""...","{\n ""AT&T"": 18,\n ""Boost Mobile"": 1,\n ""Cri...","{\n ""Amazon Prime"": 15,\n ""DirecTV"": 2,\n ""...","{\n ""Dashpass"": 1,\n ""DoorDash"": 2,\n ""Hell...","{\n ""Lyft"": 9,\n ""Uber"": 22\n}","{\n ""Affirm.com"": 2,\n ""Afterpay"": 2,\n ""Kl...","{\n ""Apple Card"": 1,\n ""Apple Cash"": 1,\n ""..."
5,223-222@63g-jj2-9mk,224-222@63g-jhf-k75,Michaels Stores,"Sporting Goods, Hobby, and Musical Instrument ...","Hobb

In [227]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 48 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   PLACEKEY                                         100 non-null    object 
 1   PARENT_PLACEKEY                                  40 non-null     object 
 2   LOCATION_NAME                                    100 non-null    object 
 3   TOP_CATEGORY                                     100 non-null    object 
 4   SUB_CATEGORY                                     100 non-null    object 
 5   NAICS_CODE                                       100 non-null    int64  
 6   LATITUDE                                         100 non-null    float64
 7   LONGITUDE                                        100 non-null    float64
 8   STREET_ADDRESS                                   100 non-null    object 
 9   CITY                             

In [355]:
# Choosing necessary columns only
new_df = df[
    [ 'PLACEKEY','CITY', 'REGION', 
     'BRANDS','SPEND_DATE_RANGE_START','SPEND_DATE_RANGE_END',
     'RAW_NUM_TRANSACTIONS','RAW_TOTAL_SPEND','RAW_NUM_CUSTOMERS',
     'SPEND_PER_TRANSACTION_BY_DAY','MEDIAN_SPEND_PER_CUSTOMER', 'SPEND_BY_DAY',
     'ONLINE_TRANSACTIONS','ONLINE_SPEND',
     'RELATED_STREAMING_CABLE_PCT','RELATED_PAYMENT_PLATFORM_PCT']
     ]
new_df.head(10)
    

,PLACEKEY,CITY,REGION,BRANDS,SPEND_DATE_RANGE_START,SPEND_DATE_RANGE_END,RAW_NUM_TRANSACTIONS,RAW_TOTAL_SPEND,RAW_NUM_CUSTOMERS,SPEND_PER_TRANSACTION_BY_DAY,MEDIAN_SPEND_PER_CUSTOMER,SPEND_BY_DAY,ONLINE_TRANSACTIONS,ONLINE_SPEND,RELATED_STREAMING_CABLE_PCT,RELATED_PAYMENT_PLATFORM_PCT
0,222-227@8gn-9sp-nqz,Tuscaloosa,AL,Ross Stores,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,290,12004.29,244,"[\n 39.1,\n 21.77,\n 27.23,\n 31.59,\n 71...",34.02,"[\n 580.9,\n 151.39,\n 125.2,\n 327.33,\n ...",0,0.00,"{\n ""Amazon Prime"": 19,\n ""Amazon Prime Vide...","{\n ""Apple Card"": 1,\n ""Apple Cash"": 1,\n ""..."
1,223-222@8f3-x8h-hkf,Hattiesburg,MS,Cracker Barrel,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,212,6355.72,181,"[\n 25.15,\n 62.12,\n 23.53,\n 27.42,\n n...",28.62,"[\n 79.99,\n 163.16,\n 91,\n 212.55,\n 0,...",0,0.00,"{\n ""Amazon Prime"": 24,\n ""Amazon Prime Vide...","{\n ""Apple Cash"": 1\n}"
2,zzw-222@8gf-ddy-6p9,Charlotte,NC,Which Wich,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,254,4111.76,209,"[\n 9.96,\n 10.23,\n 8.98,\n 30.2,\n 21.4...",14.83,"[\n 131.22,\n 20.45,\n 43.29,\n 318.01,\n ...",21,271.27,"{\n ""Amazon Prime"": 15,\n ""DirecTV"": 3,\n ""...","{\n ""Apple Card"": 2,\n ""Apple Cash"": 1,\n ""..."
3,223-222@5s5-gzd-6c5,Dutton,MI,Marathon,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,265,4745.83,100,"[\n 9.05,\n 8.59,\n 4.63,\n 9.16,\n null,...",35.00,"[\n 53.14,\n 159.54,\n 38.03,\n 72.59,\n ...",0,0.00,"{\n ""Amazon Prime"": 23,\n ""Dish Network"": 2,...","{\n ""Apple Card"": 1,\n ""Cash App"": 1,\n ""Pa..."
4,zzw-223@8fz-thj-q75,Palm Beach Gardens,FL,Starbucks,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,526,4147.79,288,"[\n 5.62,\n 6.23,\n 5.95,\n 5.78,\n null,...",8.86,"[\n 145.67,\n 134.84,\n 150.58,\n 98.82,\n...",0,0.00,"{\n ""Amazon Prime"": 15,\n ""DirecTV"": 2,\n ""...","{\n ""Apple Card"": 1,\n ""Apple Cash"": 1,\n ""..."
5,223-222@63g-jj2-9mk,Harrisonburg,VA,Michaels Stores,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,649,14475.39,505,"[\n 21.03,\n 8.42,\n 26.3,\n 20.2,\n 25.4...",18.95,"[\n 392.84,\n 111.46,\n 270.2,\n 452.43,\n...",0,0.00,"{\n ""Amazon Prime"": 19,\n ""DirecTV"": 2,\n ""...","{\n ""Apple Card"": 1,\n ""Apple Cash"": 1,\n ""..."
6,zzw-223@8dj-gqz-q2k,Lakeland,FL,Publix Super Markets,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,59,1006.83,44,"[\n 22.83,\n 59.44,\n 6.49,\n 1.89,\n nul...",10.72,"[\n 121.23,\n 59.44,\n 12.98,\n 3.78,\n 0...",0,0.00,"{\n ""Amazon Prime"": 14,\n ""DirecTV"": 3,\n ""...","{\n ""PayPal"": 10\n}"
7,225-222@63s-s8b-5mk,Taneytown,MD,Food Lion,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,3529,147252.53,1085,"[\n 23.93,\n 20.59,\n 21.28,\n 18.92,\n 3...",67.27,"[\n 4209.4,\n 4726.64,\n 3299.46,\n 3142.8...",2,76.07,"{\n ""Amazon Prime"": 18,\n ""Amazon Prime Vide...","{\n ""Apple Card"": 1,\n ""Apple Cash"": 2,\n ""..."
8,226-222@63s-9g7-rc5,Millersville,PA,Domino's Pizza,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,507,11745.26,354,"[\n 24.96,\n 17.99,\n 31.51,\n 21.33,\n 1...",26.46,"[\n 255.99,\n 413.01,\n 250.2,\n 474.33,\n...",0,0.00,"{\n ""Amazon Prime"": 21,\n ""Crunchyroll"": 2,\...","{\n ""Apple Card"": 2,\n ""Apple Cash"": 3,\n ""..."
9,zzy-222@5s7-2fv-mrk,Clarkston,MI,Jersey Mike's,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,105,1410.11,73,"[\n null,\n 9.86,\n 12.59,\n 14.53,\n nul...",15.53,"[\n 0,\n 42.56,\n 50.47,\n 77.39,\n 0,\n ...",0,0.00,"{\n ""Amazon Prime"": 20,\n ""Dish Network"": 2,...","{\n ""Apple Cash"": 5,\n ""PayPal"": 3\n}"


**There are columns with json objects.**

*Converting those columns to useful and usable types*

In [356]:
# parse the values of of certain columns into either a list or a dictionary 
def parse(x):
    if x == np.nan:
        return []
    
    if isinstance(x, (list,dict)):
        return x
    s = str(x).strip()
    if s.startswith('['):
        return json.loads(s)
    elif s.startswith('{'):
        return ast.literal_eval(s)
    else: 
        return []

# loop through each column and parse it
for col in ['SPEND_BY_DAY','RELATED_STREAMING_CABLE_PCT','RELATED_PAYMENT_PLATFORM_PCT','SPEND_PER_TRANSACTION_BY_DAY']:
    new_df[col] = new_df[col].apply(lambda x: parse(x))

new_df

/var/folders/p6/7rqrp6w17vl0ny3w4x6480br0000gn/T/ipykernel_13910/3704059017.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = new_df[col].apply(lambda x: parse(x))


,PLACEKEY,CITY,REGION,BRANDS,SPEND_DATE_RANGE_START,SPEND_DATE_RANGE_END,RAW_NUM_TRANSACTIONS,RAW_TOTAL_SPEND,RAW_NUM_CUSTOMERS,SPEND_PER_TRANSACTION_BY_DAY,MEDIAN_SPEND_PER_CUSTOMER,SPEND_BY_DAY,ONLINE_TRANSACTIONS,ONLINE_SPEND,RELATED_STREAMING_CABLE_PCT,RELATED_PAYMENT_PLATFORM_PCT
0,222-227@8gn-9sp-nqz,Tuscaloosa,AL,Ross Stores,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,290,12004.29,244,"[39.1, 21.77, 27.23, 31.59, 71.91, 157.41, 36....",34.02,"[580.9, 151.39, 125.2, 327.33, 181.93, 157.41,...",0,0.00,"{'Amazon Prime': 19, 'Amazon Prime Video': 1, ...","{'Apple Card': 1, 'Apple Cash': 1, 'Cash App':..."
1,223-222@8f3-x8h-hkf,Hattiesburg,MS,Cracker Barrel,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,212,6355.72,181,"[25.15, 62.12, 23.53, 27.42, None, 28.62, 27.0...",28.62,"[79.99, 163.16, 91, 212.55, 0, 28.62, 665.15, ...",0,0.00,"{'Amazon Prime': 24, 'Amazon Prime Video': 1, ...",{'Apple Cash': 1}
2,zzw-222@8gf-ddy-6p9,Charlotte,NC,Which Wich,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,254,4111.76,209,"[9.96, 10.23, 8.98, 30.2, 21.49, 6.44, 14.51, ...",14.83,"[131.22, 20.45, 43.29, 318.01, 42.98, 6.44, 52...",21,271.27,"{'Amazon Prime': 15, 'DirecTV': 3, 'Dish Netwo...","{'Apple Card': 2, 'Apple Cash': 1, 'Cash App':..."
3,223-222@5s5-gzd-6c5,Dutton,MI,Marathon,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,265,4745.83,100,"[9.05, 8.59, 4.63, 9.16, None, None, 13.24, 15...",35.00,"[53.14, 159.54, 38.03, 72.59, 0, 0, 441.66, 16...",0,0.00,"{'Amazon Prime': 23, 'Dish Network': 2, 'Fubo'...","{'Apple Card': 1, 'Cash App': 1, 'PayPal': 1}"
4,zzw-223@8fz-thj-q75,Palm Beach Gardens,FL,Starbucks,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,526,4147.79,288,"[5.62, 6.23, 5.95, 5.78, None, 10.33, 6.45, 4....",8.86,"[145.67, 134.84, 150.58, 98.82, 0, 10.33, 287....",0,0.00,"{'Amazon Prime': 15, 'DirecTV': 2, 'Fubo': 1, ...","{'Apple Card': 1, 'Apple Cash': 1, 'Cash App':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,22z-222@5pp-j9w-xh5,Sedalia,MO,Cost Cutters,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,5,113.40,4,"[None, 16, None, None, None, None, None, None,...",26.48,"[0, 32, 0, 0, 0, 0, 0, 0, 0, 43.45, 0, 0, 0, 0...",1,17.00,"{'Amazon Prime': 25, 'Hulu': 50, 'Netflix': 25...",{}
96,zzy-222@645-wz2-dvz,Florence,SC,Waffle House,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,52,1012.69,51,"[None, 6.05, 23.96, 27.51, None, None, 14.89, ...",18.70,"[0, 6.05, 23.96, 55.02, 0, 0, 157.38, 53.08, 7...",0,0.00,"{'Amazon Prime': 28, 'Dish Network': 4, 'Hulu'...",{'PayPal': 4}
97,zzw-222@63s-cqy-m8v,West Chester,PA,Smoothie King,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,295,2310.02,198,"[5.82, 12.17, 7.19, 6.14, 6.14, None, 8.05, 8....",8.15,"[59.47, 49.53, 73.55, 126.02, 6.14, 0, 303.56,...",0,0.00,"{'Amazon Prime': 15, 'Crunchyroll': 1, 'Hulu':...","{'Apple Card': 1, 'PayPal': 2, 'Venmo': 1}"
98,222-222@5vg-s6m-hnq,Elk Grove,CA,Walgreens,2019-10-01T00:00:00-07:00,2019-11-01T00:00:00-07:00,247,6351.03,143,"[20.78, 23.13, 11.3, 25.57, 26.93, 29.64, 10.2...",25.83,"[120.65, 283.92, 179.05, 204.14, 53.86, 59.27,...",0,0.00,"{'Amazon Prime': 21, 'DirecTV': 3, 'Dish Netwo...","{'Cash App': 1, 'PayPal': 3}"


In [357]:
# Create new columns with the tuples of the messy values of those two columns 

new_df['streaming_services'] = new_df['RELATED_STREAMING_CABLE_PCT'].apply(lambda x: tuple(x.items()) if isinstance(x, dict) else [])
new_df['payment_methods'] = new_df['RELATED_PAYMENT_PLATFORM_PCT'].apply(lambda x: list(x.items()) if isinstance(x, dict) else [])
new_df['Spend_Per_transaction_by_day'] = new_df['SPEND_PER_TRANSACTION_BY_DAY'].apply(lambda lst: [0 if x == None else x for x in lst])

new_df

/var/folders/p6/7rqrp6w17vl0ny3w4x6480br0000gn/T/ipykernel_13910/1711910507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['streaming_services'] = new_df['RELATED_STREAMING_CABLE_PCT'].apply(lambda x: tuple(x.items()) if isinstance(x, dict) else [])
/var/folders/p6/7rqrp6w17vl0ny3w4x6480br0000gn/T/ipykernel_13910/1711910507.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['payment_methods'] = new_df['RELATED_PAYMENT_PLATFORM_PCT'].apply(lambda x: list(x.items()) if isinstance(x, dict) 

,PLACEKEY,CITY,REGION,BRANDS,SPEND_DATE_RANGE_START,SPEND_DATE_RANGE_END,RAW_NUM_TRANSACTIONS,RAW_TOTAL_SPEND,RAW_NUM_CUSTOMERS,SPEND_PER_TRANSACTION_BY_DAY,MEDIAN_SPEND_PER_CUSTOMER,SPEND_BY_DAY,ONLINE_TRANSACTIONS,ONLINE_SPEND,RELATED_STREAMING_CABLE_PCT,RELATED_PAYMENT_PLATFORM_PCT,streaming_services,payment_methods,Spend_Per_transaction_by_day
0,222-227@8gn-9sp-nqz,Tuscaloosa,AL,Ross Stores,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,290,12004.29,244,"[39.1, 21.77, 27.23, 31.59, 71.91, 157.41, 36....",34.02,"[580.9, 151.39, 125.2, 327.33, 181.93, 157.41,...",0,0.00,"{'Amazon Prime': 19, 'Amazon Prime Video': 1, ...","{'Apple Card': 1, 'Apple Cash': 1, 'Cash App':...","((Amazon Prime, 19), (Amazon Prime Video, 1), ...","[(Apple Card, 1), (Apple Cash, 1), (Cash App, ...","[39.1, 21.77, 27.23, 31.59, 71.91, 157.41, 36...."
1,223-222@8f3-x8h-hkf,Hattiesburg,MS,Cracker Barrel,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,212,6355.72,181,"[25.15, 62.12, 23.53, 27.42, None, 28.62, 27.0...",28.62,"[79.99, 163.16, 91, 212.55, 0, 28.62, 665.15, ...",0,0.00,"{'Amazon Prime': 24, 'Amazon Prime Video': 1, ...",{'Apple Cash': 1},"((Amazon Prime, 24), (Amazon Prime Video, 1), ...","[(Apple Cash, 1)]","[25.15, 62.12, 23.53, 27.42, 0, 28.62, 27.06, ..."
2,zzw-222@8gf-ddy-6p9,Charlotte,NC,Which Wich,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,254,4111.76,209,"[9.96, 10.23, 8.98, 30.2, 21.49, 6.44, 14.51, ...",14.83,"[131.22, 20.45, 43.29, 318.01, 42.98, 6.44, 52...",21,271.27,"{'Amazon Prime': 15, 'DirecTV': 3, 'Dish Netwo...","{'Apple Card': 2, 'Apple Cash': 1, 'Cash App':...","((Amazon Prime, 15), (DirecTV, 3), (Dish Netwo...","[(Apple Card, 2), (Apple Cash, 1), (Cash App, ...","[9.96, 10.23, 8.98, 30.2, 21.49, 6.44, 14.51, ..."
3,223-222@5s5-gzd-6c5,Dutton,MI,Marathon,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,265,4745.83,100,"[9.05, 8.59, 4.63, 9.16, None, None, 13.24, 15...",35.00,"[53.14, 159.54, 38.03, 72.59, 0, 0, 441.66, 16...",0,0.00,"{'Amazon Prime': 23, 'Dish Network': 2, 'Fubo'...","{'Apple Card': 1, 'Cash App': 1, 'PayPal': 1}","((Amazon Prime, 23), (Dish Network, 2), (Fubo,...","[(Apple Card, 1), (Cash App, 1), (PayPal, 1)]","[9.05, 8.59, 4.63, 9.16, 0, 0, 13.24, 15.16, 1..."
4,zzw-223@8fz-thj-q75,Palm Beach Gardens,FL,Starbucks,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,526,4147.79,288,"[5.62, 6.23, 5.95, 5.78, None, 10.33, 6.45, 4....",8.86,"[145.67, 134.84, 150.58, 98.82, 0, 10.33, 287....",0,0.00,"{'Amazon Prime': 15, 'DirecTV': 2, 'Fubo': 1, ...","{'Apple Card': 1, 'Apple Cash': 1, 'Cash App':...","((Amazon Prime, 15), (DirecTV, 2), (Fubo, 1), ...","[(Apple Card, 1), (Apple Cash, 1), (Cash App, ...","[5.62, 6.23, 5.95, 5.78, 0, 10.33, 6.45, 4.55,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,22z-222@5pp-j9w-xh5,Sedalia,MO,Cost Cutters,2019-10-01T00:00:00-05:00,2019-11-01T00:00:00-05:00,5,113.40,4,"[None, 16, None, None, None, None, None, None,...",26.48,"[0, 32, 0, 0, 0, 0, 0, 0, 0, 43.45, 0, 0, 0, 0...",1,17.00,"{'Amazon Prime': 25, 'Hulu': 50, 'Netflix': 25...",{},"((Amazon Prime, 25), (Hulu, 50), (Netflix, 25)...",[],"[0, 16, 0, 0, 0, 0, 0, 0, 0, 43.45, 0, 0, 0, 0..."
96,zzy-222@645-wz2-dvz,Florence,SC,Waffle House,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,52,1012.69,51,"[None, 6.05, 23.96, 27.51, None, None, 14.89, ...",18.70,"[0, 6.05, 23.96, 55.02, 0, 0, 157.38, 53.08, 7...",0,0.00,"{'Amazon Prime': 28, 'Dish Network': 4, 'Hulu'...",{'PayPal': 4},"((Amazon Prime, 28), (Dish Network, 4), (Hulu,...","[(PayPal, 4)]","[0, 6.05, 23.96, 27.51, 0, 0, 14.89, 26.54, 18..."
97,zzw-222@63s-cqy-m8v,West Chester,PA,Smoothie King,2019-10-01T00:00:00-04:00,2019-11-01T00:00:00-04:00,295,2310.02,198,"[5.82, 12.17, 7.19, 6.14, 6.14, None, 8.05, 8....",8.15,"[59.47, 49.53, 73.55, 126.02, 6.14, 0, 303.56,...",0,0.00,"{'Amazon Prime': 15, 'Crunchyroll': 1, 'Hulu':...","{'Apple Card': 1, 'PayPal': 2, 'Venmo': 1}","((Amazon Prime, 15), (Crunchyroll

In [360]:
# There is no need for those columns anymore

new_df.drop(columns = ['RELATED_STREAMING_CABLE_PCT','RELATED_PAYMENT_PLATFORM_PCT','SPEND_DATE_RANGE_END','SPEND_PER_TRANSACTION_BY_DAY'], errors = 'ignore', inplace=True)

/var/folders/p6/7rqrp6w17vl0ny3w4x6480br0000gn/T/ipykernel_13910/2351676723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns = ['RELATED_STREAMING_CABLE_PCT','RELATED_PAYMENT_PLATFORM_PCT','SPEND_DATE_RANGE_END','SPEND_PER_TRANSACTION_BY_DAY'], errors = 'ignore', inplace=True)


*Changing the date columns to a datetime format*

In [361]:
# Change the start date column to a datetime type
new_df['SPEND_DATE_RANGE_START'] = pd.to_datetime(
    new_df['SPEND_DATE_RANGE_START'],
    format='%Y-%m-%dT%H:%M:%S%z',
    utc=True             
).dt.tz_convert(None) 

# Extract the Year and Month
# year_month column contains the start month of the data and ends at the start of the next month
new_df['year_month'] = new_df['SPEND_DATE_RANGE_START'].dt.strftime('%Y-%m')
col = new_df.pop('year_month')
new_df.insert(5, col.name, col)

# sort the values based on date and drop the unecessary column
df_cleaned = new_df.sort_values(['year_month']).reset_index(drop = True)
df_cleaned.drop(columns = ['SPEND_DATE_RANGE_START'], errors = 'ignore', inplace=True)

df_cleaned

/var/folders/p6/7rqrp6w17vl0ny3w4x6480br0000gn/T/ipykernel_13910/514431599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['SPEND_DATE_RANGE_START'] = pd.to_datetime(
/var/folders/p6/7rqrp6w17vl0ny3w4x6480br0000gn/T/ipykernel_13910/514431599.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['year_month'] = new_df['SPEND_DATE_RANGE_START'].dt.strftime('%Y-%m')


,PLACEKEY,CITY,REGION,BRANDS,year_month,RAW_NUM_TRANSACTIONS,RAW_TOTAL_SPEND,RAW_NUM_CUSTOMERS,MEDIAN_SPEND_PER_CUSTOMER,SPEND_BY_DAY,ONLINE_TRANSACTIONS,ONLINE_SPEND,streaming_services,payment_methods,Spend_Per_transaction_by_day
0,zzy-22k@627-xzp-zpv,Hicksville,NY,Avis Rent A Car,2019-09,4,465.82,4,111.47,"[75.98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14...",0,0.00,"((Netflix, 25),)",[],"[75.98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14..."
1,zzw-222@5p9-9yq-py9,Mason City,IA,SmartStyle Family Hair Salons,2019-09,4,124.17,4,31.06,"[0, 0, 0, 20.09, 0, 0, 0, 0, 84.08, 0, 0, 0, 2...",0,0.00,"((DirecTV, 25), (Hulu, 25), (Netflix, 50), (Re...","[(PayPal, 50)]","[0, 0, 0, 20.09, 0, 0, 0, 0, 42.04, 0, 0, 0, 2..."
2,222-222@8gj-8xw-qcq,Tallahassee,FL,Walgreens,2019-09,260,5402.72,173,17.19,"[39.97, 0, 713.94, 40.52, 163.61, 219.74, 87.4...",0,0.00,"((Amazon Prime, 13), (DirecTV, 2), (Dish Netwo...","[(Apple Card, 1), (Apple Cash, 1), (PayPal, 3)]","[12.99, 0, 17.02, 7.99, 13.18, 17.57, 43.72, 1..."
3,zzw-223@627-s6h-y7q,New York,NY,T.J. Maxx,2019-09,900,42544.40,713,39.57,"[161.08, 21.75, 4775.02, 1219, 1155.51, 1275.1...",1,53.09,"((Amazon Prime, 14), (DirecTV, 1), (Dish Netwo...","[(Apple Card, 1), (Apple Cash, 1), (Cash App, ...","[80.54, 21.75, 28.38, 40.26, 32.14, 24.41, 4.8..."
4,zzw-223@5qw-62w-f9f,Kaufman,TX,McDonald's,2019-09,333,3453.91,203,11.45,"[17.93, 11.24, 435.95, 94.08, 42.19, 110.58, 2...",0,0.00,"((Amazon Prime, 18), (Amazon Prime Video, 1), ...","[(Apple Card, 1), (Apple Cash, 1), (PayPal, 5)]","[8.97, 11.24, 7.55, 7.89, 6.97, 10.15, 6.5, 3...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,225-222@8fx-yv7-b6k,Vero Beach,FL,Hungry Howie's Pizza & Subs,2020-07,113,2719.73,79,28.71,"[74.04, 140.55, 129.18, 0, 0, 300.75, 10.63, 3...",0,0.00,"((Amazon Prime Video, 2), (Disney Plus, 7), (H...","[(Apple Card, 2), (Apple Cash, 3)]","[22.46, 14.29, 64.59, 0, 0, 21.23, 10.63, 35.2..."
96,225-222@63v-6kw-6ff,Westlake,OH,Once Upon a Child,2020-07,63,2519.39,62,29.70,"[0, 338.58, 14.04, 0, 0, 233.28, 166.86, 113.9...",0,0.00,"((Amazon Prime, 7), (Amazon Prime Video, 9), (...","[(Cash App, 2)]","[0, 56.7, 14.04, 0, 0, 25.38, 26.19, 29.7, 0, ..."
97,zzw-224@8gg-szd-8n5,Greenville,SC,Christmas Tree Shops,2020-07,285,11363.03,237,34.30,"[444.85, 315.79, 693.93, 384.85, 295.53, 822.4...",0,0.00,"((Amazon Prime Video, 1), (Crunchyroll, 1), (D...","[(Apple Card, 1), (Apple Cash, 3), (Cash App, ...","[29.77, 24.28, 32.78, 189.04, 29.59, 29.51, 25..."
98,223-222@5qw-t3h-x3q,Dallas,TX,McDonald's,2020-07,301,2998.63,177,11.01,"[94.16, 94.79, 133.36, 22.01, 15.11, 321.41, 9...",0,0.00,"((Amazon Prime Video, 1), (Crunchyroll, 1), (D...","[(Apple Card, 2), (Apple Cash, 3), (PayPal, 3)]","[9.73, 5.73, 10.09, 3.99, 7.56, 13.78, 7.51, 6..."


# Questions

1. **Is there a correlation between states that have high spending per customer with the customers use of Apple payment methods?**
    - How does getting older change purchase method and amount? Do older people choose a specific method like cash payment? 
    - Do older people tend to apply more discounts? (testing an assumption)
2. **here**

    -

---------